<a href="https://colab.research.google.com/github/Trisha11r/MozillaOpenLab/blob/master/MozillaLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Drive Mount code

from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# %cd gdrive/My Drive/Mozilla
%ls

In [0]:
#@title Import Packages
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
from nltk import word_tokenize 
from nltk.util import ngrams
from collections import Counter
import pickle
from joblib import dump, load

import sklearn
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB

from sklearn import metrics
from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer



for reference:

Labels:
0=ND, 1=D

Donation_Related:
0=request, 1=offer, -1 = N/A

Resource_Type:
0=Money, 1=Clothing, 2=Food, 3=Medical, 4=Shelter, 5=Volunteer, -1 = N/A

In [0]:
#@title Stop Words
stop_words = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are",
                     "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both",
                     "but", "by", "can't", "cannot", "could", "couldn't", "did", "didn't", "do", "does", "doesn't",
                     "doing", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn't",
                     "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll", "he's", "her", "here",
                     "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm",
                     "i've", "if", "in", "into", "is", "isn't", "it", "it's", "its", "itself", "let's", "me", "more"
                     , "mustn't", "my", "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or",
                     "ought", "our", "ours    ourselves", "out", "over", "own", "same", "shan't", "she",
                     "she'd", "she'll", "she's", "should", "shouldn't", "so", "some", "such", "than", "that", "that's",
                     "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they",
                     "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under",
                     "until", "up", "very", "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't",
                     "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom",
                     "why", "why's", "with", "won't", "would", "wouldn't", "you", "you'd", "you'll", "you're", "you've",
                     "your", "yours", "yourself", "yourselves", 'a', 'about', 'above', 'across', 'after', 'again',
                     'against', 'all', 'almost', 'alone', 'btw', 'north', 'south', 'east', 'west', 'sarita', 'woke', 'wake',
                     'suv', 'omg', 'asap', 'contain', 'au', 'demi', 'mam', 'sir', "ma'am", "i'm'", 'ohh', 'oh', 'duh',
                     'go', 'goes', 'went', 'gone', 'dollar', 'dollars', 'cents', 'cent', 'usa', 'dont', 'aaa',
                     'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any',
                     'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask',
                     'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be',
                     'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being',
                     'beings', 'between', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'couldnt',
                     'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'coz', 'd', 'did',
                     'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'downed', 'downing', 'downs',
                     'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even',
                     'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f',
                     'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'first', 'for', 'four', 'from',
                     'full', 'fully', 'further', 'furthered', 'furthering', 'furthers', 'g', 'gave', 'general',
                     'generally', 'get', 'gets', 'give', 'given', 'gives', 'go', 'going', 'goods', 'got',
                     'group', 'grouped', 'grouping', 'groups', 'h', 'had', 'has',
                     'have', 'having', 'he', 'her', 'here', 'herself',
                     'him', 'himself', 'his', 'how', 'however', 'i', 'if', 'important', 'in', 'into', 'is', 'it', 'its',
                     'itself', 'j', 'just', 'k', 'keep', 'keeps',
                     'knew', 'know', 'known', 'knows', 'l', 'largely', 'later', 'latest',
                     'least', 'let', 'lets', 'likely', 'm', 'made', 'make',
                     'making', 'man', 'many', 'may', 'me', 'member', 'members', 'men', 'might', 'more', 'most',
                     'mostly', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 'n', 'necessary', 'need', 'needed',
                     'needing', 'needs', 'new', 'next',
                     'noone', 'nothing', 'now', 'nowhere', 'number', 'numbers', 'o', 'of', 'off', 'often',
                     'old', 'older', 'oldest', 'on', 'once', 'one', 'only', 'open', 'opened', 'opening', 'opens', 'or',
                     'order', 'ordered', 'ordering', 'orders', 'other', 'others', 'our', 'out', 'over', 'p', 'part',
                     'parted', 'parting', 'parts', 'per', 'perhaps', 'place', 'places', 'point', 'pointed', 'pointing',
                     'points', 'possible', 'present', 'presented', 'presenting', 'presents',
                     'put', 'puts', 'q', 'quite', 'r', 'rather', 'really', 'right', 'room', 'rooms', 's',
                     'said', 'same', 'saw', 'say', 'says', 'second', 'seconds', 'see', 'seem', 'seemed', 'seeming',
                     'seems', 'sees', 'several', 'shall', 'she', 'should', 'show', 'showed', 'showing', 'shows', 'side',
                     'sides', 'since', 'so', 'some', 'somebody', 'someone', 'something',
                     'somewhere', 'state', 'states', 'still', 'such', 'sure', 't', 'take', 'taken', 'than',
                     'that', 'the', 'their', 'them', 'then', 'there', 'therefore', 'these', 'they', 'thing', 'things',
                     'think', 'thinks', 'this', 'those', 'though', 'thought', 'thoughts', 'three', 'through', 'thus',
                     'to', 'today', 'together', 'too', 'took', 'toward', 'turn', 'turned', 'turning', 'turns', 'two',
                     'u', 'under', 'until', 'up', 'upon', 'us', 'use', 'used', 'uses', 'v', 'very', 'w', 'want',
                     'wanted', 'wanting', 'wants', 'was', 'way', 'ways', 'we', 'We', 'well', 'wells', 'went', 'were', 'what',
                     'when', 'where', 'whether', 'which', 'while', 'who', 'whole', 'whose', 'why', 'will', 'with',
                     'within', 'without', 'work', 'worked', 'working', 'works', 'would', 'x', 'y', 'year', 'years',
                     'yet', 'you', 'young', 'younger', 'youngest', 'your', 'yours', 'z', 'weren', 'didn', 'ours', 'hasn', 'hadn', "should've", 'ourselves', 're', 'wouldn', 've', 'ain', 'couldn', 'mustn', 'aren', 'isn', 'wasn', 'doesn', 'll', "that'll", 'mightn', 'won', 'shan', "mightn't", "needn't", 'haven', 'needn', 'ma', 'don', 'shouldn']

In [0]:
#@title Clean Data

# -Removing non–ASCII characters.
# -Separating text into tokens (words), removing stopwords and performing stemming (reducing to root words, such as ‘helping’ to ‘help’).
# -Generalizing some tokens by replacing numbers by the token _NUM_, hyperlinks by the token _URL_, retweets (“RT @user_name”) by the token _RT_ and lastly, user mentions in the tweets (@user_name) by the token _MENTION_.
# -I/P = data to be cleaned
# -O/P = return cleaned data

# flag = 0 -> For cleaning tweets for training
# flag = 1 -> For cleaning tweets to show on website
def clean(data, flag):
	
	ps = PorterStemmer()
	lem = WordNetLemmatizer()

	data = data.replace(np.nan, 'Unknown', regex=True)

	if flag==1:
		data['Tweet_cleaned'] = ""
	else:
		data['Tweet'] = ""
	

	for index, row in data.iterrows():
		s = ''
		row_tweet = row['Actual_Tweet'].replace('\\n', ' ')
		tweet = row_tweet.split(' ')

		urlFound = False

		for i, c in enumerate(tweet):
			if( c.startswith('b') and c.endswith('RT') ):
				continue
			elif( c.startswith('@') and c.endswith(':') ):
				continue
			elif ( 'http' in c ):
				urlFound = True
				
				t = ""
				for x in c:
					if ( (x>='a' and x<='z') or (x>='A' and x<='Z') or (x>='0' and x<='9') or x==':' or x=='/' or x=="."):
						t += x
					elif x=='\\':
						break

				data.at[index, 'URL'] = t
				continue
			elif ( re.search("^.*x[a-z][0-9].*$" , c) ):
				continue
			elif ( c.startswith('#') or c.startswith('@')):
				if flag==1 and c.startswith('#'):
					s += c + ' '
				else:
					continue

			else:
				if( i==0 and (c.startswith('b\'') or c.startswith('b\"') ) ):
					c = c[2:]

				if flag==0:
					c = c.lower()

					# Removing Stop Words using nltk
					if c in stop_words:
						continue
				
				t = ''
				ind = -1
				count = -1

				for x in c:
					
					ind += 1
					if flag==0 and x=='#':
						break

					if ( (x>='a' and x<='z') or (x>='A' and x<='Z') ):
						t += x
						count = 0
					elif count==0:
						t += ' '
						count = -1

				if flag==0:
					t = lem.lemmatize(t, pos='v')
					if t in stop_words:
						continue

				s += t + ' '

		
		# if s=='':
		# 	print (index)

		if flag==0:
			# Removing one, two letter words
			s = re.sub(r'\b\w{1,2}\b', '', s)

		# Remove extra whitespaces in between
		s = " ".join(s.split())
		
		if flag==0:
			row['Tweet'] = s
		else:
			data.at[index, 'Tweet_cleaned'] = s
			if urlFound==False:
				data.at[index, 'URL'] = 'Not Available'
	
	if flag==0:	
		data = data.drop_duplicates(subset=['Tweet'], keep='first')
		data['Tweet'].replace('', np.nan, inplace=True)
		data.dropna(subset=['Tweet'], inplace=True)
	
	print ('Data cleaned')

	return data

In [0]:
nltk.download('wordnet')
# Clean tweets for training purpose [flag=0] - Performing Stemming, removing one or two letter words
cleanData = clean(data, 0)
# Clean tweets for Display purpose [flag=1] - Avoiding Stemming & removing one or two letter words
data = clean(cleanData, 1)
data.to_csv('final_dataset_cleaned.csv', index=False)

In [0]:
data = pd.read_csv('final_dataset_cleaned.csv')

In [0]:
#@title Load Models
loaded_models = {}
classes = ['Labels', 'Donation_Related', 'Resource_Type']

for cls in classes:
  
  if cls=='Labels':
    print ('Loading Model for Donation/Non-Donation...')

    loaded_models['d_nd'] = load("models/donation_nonDonation.joblib")

  elif cls=='Donation_Related':
    print ('Loading Model for Only Donation Data- Request/Offer...')

    loaded_models['req_off'] = load("models/request_offer.joblib")
  else:
    print ('Loading Model for Only Donation Data- Resource Type...')

    loaded_models['res_type'] = load("models/resource_type.joblib")

In [0]:
#@title Prediction

data['D_ND'] = loaded_models['d_nd'].predict(data['Tweet'])
data['Request_Offer'] = loaded_models['req_off'].predict(data['Tweet'])
data['Resource_Type'] = loaded_models['res_type'].predict(data['Tweet'])

data.loc[data['D_ND']==0, ['Request_Offer', 'Resource_Type']] = -1

data.to_csv('final_dataset_24k.csv', index=False)


In [0]:
# Old Dataset - 3k tweets
data1 = pd.read_csv('final_dataset_3k.csv')
# Old Dataset - 24k tweets
data2 = pd.read_csv('final_dataset_24k.csv')

data = pd.concat([data1, data2])
data.to_csv('final_dataset.csv', index=False)

In [0]:
#@title Create uni, bi, tri, quad and penta tokens from sentences

def create_ngramtokens(df):
  
  tweets = df['Tweet']

  # Donation/Non-Donation
  d_nd = df['D_ND']

  # Request/Offer
  ro = df['Request_Offer']

  # Resource Type
  typ = df['Resource_Type']
  

  features = []
  don_nod = []
  req_off = []
  res_type = []

  print (len(ro))
  # i = 0
  for ind, tweet in enumerate(tweets):
    tweetToken = nltk.word_tokenize(tweet)

    for len_grams in range(1, 5):

      ngram = ngrams(tweetToken, len_grams)
      for gram in ngram:
        features.append([g for g in gram])
        # print (ind)
        don_nod.append(int(float(d_nd[ind])))
        req_off.append(int(float(ro[ind])))
        res_type.append(int(float(typ[ind])))
    
    # i += 1
 
  data = pd.DataFrame(columns=['Features', 'D_ND', 'Request_Offer', 'Resource_Type'])

  data['Features'] = features
  data['D_ND'] = don_nod
  data['Request_Offer'] = req_off
  data['Resource_Type'] = res_type
  

  data.to_csv('training_data.csv', index=False)

  data[data['D_ND']==1].to_csv('training_data_donation.csv', index=False)


In [0]:
#@title Training Model using Multinomial Naive Bayes
def train():
  classes = ['D_ND', 'Request_Offer', 'Resource_Type']

  for cls in classes:

    
    if cls=='D_ND':
      data = pd.read_csv('training_data.csv')
      print ('Training Model for Donation/Non-Donation')
    else:
      data = pd.read_csv('training_data_donation.csv')
      if cls=='Request_Offer':
        print ('Training Model for Only Donation Data- Request/Offer')
      else:
        print ('Training Model for Only Donation Data- Resource Type')

    # Split into training, testing
    X_train, X_test, y_train, y_test = train_test_split(data['Features'], data[cls], random_state = 0, test_size = 0.25)

    
    # define the stages of the pipeline
    pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True)),
                                ('model', MultinomialNB())])

    # fit the pipeline model with the training data                            
    pipeline.fit(X_train, y_train)

    # print (pipeline.predict(['Want to Donate PPE.']))
    y_pred = pipeline.predict(X_test)
    
    if cls=='D_ND':
      print ('Accuracy (Donation/Non-Donation): ', accuracy_score(y_test, y_pred))
      # dump the pipeline model
      dump(pipeline, filename="models/donation_nonDonation.joblib")
      
    elif cls=='Request_Offer':
      print ('Accuracy (Request/Offer): ', accuracy_score(y_test, y_pred))
      dump(pipeline, filename="models/request_offer.joblib")
      
    else:
      print ('Accuracy (Resource Type): ', accuracy_score(y_test, y_pred))
      dump(pipeline, filename="models/resource_type.joblib")
      
    
    print ()

In [0]:
# Now, train the model using the whole dataset

data = pd.read_csv('final_dataset.csv')
nltk.download('punkt')
create_ngramtokens(data)
train()